In [1]:
import scanpy as sc
import numpy as np
import pandas as pd

sc.settings.verbosity = 3
sc.logging.print_header()
sc.settings.set_figure_params(facecolor='white', frameon=False)

In [2]:
from pangeapy.cell import CellAnnotator
from pangeapy.meta import MetaAnnotator

In [3]:
cell_anno = CellAnnotator()
meta_anno = MetaAnnotator()

Checking-up download status of anno_models
Checking-up download status of meta_models
